# Table of Contents

1. [Setup](#setup)
2. [LPIPS Score](#lpips-score)
3. [Conditional Inception Score (CIS)](#conditional-inception-score-(cis))
4. [ArtFID Score](#artfid-score)
5. [Experiments](#experiments)
  - [edges2shoes_shoe](#edges2shoes_shoe)
  - [edges2handbags](#edges2handbags)
  - [summer 2 winter](#summer-2-winter)
  - [cat to dog](#cat-to-dog)  
6. [Appendix](#appendix)

## Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive/',force_remount=True)
# !pwd MyDrive/Classes/10-623GenAI/project
# %cd drive/MyDrive/Classes/10-623GenAI/project
!pwd MyDrive/temp
%cd drive/MyDrive/temp

Mounted at /content/drive/
/content
/content/drive/MyDrive/temp


In [ ]:
!ls

data.pt  MUNIT	MUNIT-Enhanced	prompt2prompt.py  __pycache__  requirements.txt  run_in_colab.ipynb


In [ ]:
!pip install lpips
!pip install torch torchvision numpy scipy pillow torchfile


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torchfile: filename=torchfile-0.1.0-py3-none-any.whl size=5693 sha256=66e601d5a7bbca719222623579ba3864356ac4239dd01d2556da041630d0545e
  Stored in directory: /root/.cache/pip/wheels/c7/e9/87/1c51daf8e468d5c14931f8ac3344880f903ba96b063675cac2
Successfully built torchfile


## **LPIPS Score**
Definition: LPIPS (Learned Perceptual Image Patch Similarity) measures perceptual similarity between two images. It evaluates how "perceptually" similar the generated image is to the reference, rather than pixel-wise differences.\
Value Interpretation:\
LPIPS scores range from 0 to 1, where lower values indicate higher similarity (i.e., better).

In [ ]:
import torch
import lpips
from PIL import Image
import torchvision.transforms as transforms

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Preprocess images
def preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((256, 256)),  # Resize to model's input size
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5])
    ])
    image = Image.open(image_path).convert('RGB')
    return transform(image).unsqueeze(0).to(device)  # Add batch dimension

def lpips_function(img1 = 'MUNIT/inputs/edges2shoes_shoe.jpg', img2 = 'MUNIT/results/output000.jpg'):
  # Load the pre-trained LPIPS model
  lpips_model = lpips.LPIPS(net='alex')  # Options: 'alex', 'vgg', 'squeeze'
  lpips_model.to(device).eval()

  # Load and preprocess two images
  image1_path = img1
  image2_path = img2
  image1 = preprocess_image(image1_path)
  image2 = preprocess_image(image2_path)

  # Compute LPIPS similarity
  with torch.no_grad():
      distance = lpips_model(image1, image2)
  # print(f'LPIPS Distance: {distance.item()}')
  return distance.item()

Using device: cuda


## **Conditional Inception Score (CIS)**
Definition: CIS measures how well a model generates diverse outputs for a given condition while ensuring each output is realistic. It assesses the balance between quality and conditional variability.\
Value Interpretation:\
Higher CIS values are better.\
Common Values: For strong conditional image models, CIS is typically in the range of 4–8.

In [ ]:
import torch
import torchvision.transforms as transforms
from torchvision.models import inception_v3
from scipy.stats import entropy
import numpy as np
from PIL import Image

# Load the pre-trained InceptionV3 model
inception_model = inception_v3(pretrained=True, transform_input=False)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inception_model.eval().cuda()  # Move to GPU if available

# Preprocessing function
def preprocess_image2(image_path):
    transform = transforms.Compose([
        transforms.Resize((299, 299)),  # Required input size for InceptionV3
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize to [-1, 1]
    ])
    image = Image.open(image_path).convert("RGB")
    return transform(image).unsqueeze(0).cuda()  # Add batch dimension and move to GPU

# Function to calculate softmax probabilities
def get_inception_probs(images, model):
    with torch.no_grad():
        preds = model(images)
        return torch.nn.functional.softmax(preds, dim=1).cpu().numpy()

# Conditional Inception Score Calculation
def calculate_cis(input_images, generated_images, inception_model, num_splits=5):
    """
    input_images: List of input shoe images (paths).
    generated_images: List of lists, where each sublist contains paths of generated images conditioned on the corresponding input image.
    """
    assert len(input_images) == len(generated_images), "Each input image must have corresponding generated images."
    scores = []

    for input_image, gen_images in zip(input_images, generated_images):
        # Preprocess the input image and all generated images
        input_img_tensor = preprocess_image2(input_image)
        generated_img_tensors = torch.cat([preprocess_image2(img) for img in gen_images], dim=0)

        # Get inception probabilities for the input image and generated images
        input_probs = get_inception_probs(input_img_tensor, inception_model)
        gen_probs = get_inception_probs(generated_img_tensors, inception_model)

        # Calculate Conditional Inception Score for the current set
        p_y = np.mean(gen_probs, axis=0)
        split_scores = []
        split_scores_is = []

        for i in range(num_splits):
            part = gen_probs[i * len(gen_probs) // num_splits:(i + 1) * len(gen_probs) // num_splits]
            scores_i = [entropy(p_y, p) for p in part]
            split_scores.append(np.exp(np.mean(scores_i)))
            p_y = np.mean(part, axis=0)  # Mean class probabilities
            kl_div = part * (np.log(part + 1e-10) - np.log(p_y + 1e-10))

        split_scores_is.append(np.exp(np.mean(np.sum(kl_div, axis=1))))
        scores.append(np.mean(split_scores))

    return np.mean(scores), np.mean(split_scores_is), np.std(split_scores_is)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## ArtFID Score
**FID Score**
Definition: FID (Fréchet Inception Distance) evaluates the quality and diversity of generated images by comparing the statistics of real and generated image features extracted from a pretrained Inception network.\
Value Interpretation:\
Lower values indicate the generated images are closer in distribution to the real images.

**ArtFID Score**
Definition: ArtFID is a variation of FID tailored for comparing style-transferred images, assessing both perceptual quality and alignment with artistic intent.\
Value Interpretation:\
Higher values suggest significant deviation from the input image






In [ ]:
import numpy as np
import torch
from scipy.linalg import sqrtm
from torchvision import models, transforms
from torch.utils.data import DataLoader
from PIL import Image
import os

# Function to load images and preprocess
def load_images(image_paths, transform):
    images = []
    print(image_paths)
    for path in image_paths:
        img = Image.open(path).convert("RGB")
        img = transform(img)
        images.append(img)
    return torch.stack(images)

# Function to calculate FID
def calculate_fid(real_features, generated_features):
    if real_features.ndim != 2 or generated_features.ndim != 2:
        raise ValueError("Input feature arrays must be 2D.")

    if real_features.shape[0] < 2 or generated_features.shape[0] < 2:
        raise ValueError("At least two samples are required in each feature array.")
    # Calculate means and covariances
    mu_real, sigma_real = real_features.mean(axis=0), np.cov(real_features, rowvar=False)
    mu_gen, sigma_gen = generated_features.mean(axis=0), np.cov(generated_features, rowvar=False)

    # Calculate squared difference of means
    diff = mu_real - mu_gen

    # Product of covariance matrices
    covmean, _ = sqrtm(sigma_real @ sigma_gen, disp=False)

    # Handle numerical instability
    if np.iscomplexobj(covmean):
        covmean = covmean.real

    # FID formula
    fid = diff @ diff + np.trace(sigma_real + sigma_gen - 2 * covmean)
    return fid

# Function to extract features using InceptionV3
def extract_features(image_batch, model, device):
    model.eval()
    with torch.no_grad():
        image_batch = image_batch.to(device)
        features = model(image_batch).squeeze().cpu().numpy()
    return features

# ARTFID Extension Example
def calculate_artfid(real_features, generated_features, weights):
    fid = calculate_fid(real_features, generated_features)
    artfid = fid * weights
    return artfid

# Main FID computation
def compute_fid_and_artfid(real_image_paths, generated_image_paths, weights=None, batch_size=16, device="cuda"):
    # Image transformations
    transform = transforms.Compose([
        transforms.Resize((299, 299)),  # InceptionV3 expects 299x299
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    # Load images
    real_images = load_images(real_image_paths, transform)
    generated_images = load_images(generated_image_paths, transform)

    # DataLoaders
    real_loader = DataLoader(real_images, batch_size=batch_size, shuffle=False)
    gen_loader = DataLoader(generated_images, batch_size=batch_size, shuffle=False)

    # Load InceptionV3 model (pre-trained)
    model = models.inception_v3(pretrained=True, transform_input=False).to(device)

    # Extract features
    real_features = []
    for batch in real_loader:
        real_features.append(extract_features(batch, model, device))
    real_features = np.concatenate(real_features, axis=0)

    generated_features = []
    for batch in gen_loader:
        generated_features.append(extract_features(batch, model, device))
    generated_features = np.concatenate(generated_features, axis=0)
    print(f"real_features: {real_features.shape}")

    # Calculate FID
    fid = calculate_fid(real_features, generated_features)

    # Calculate ARTFID
    artfid = None
    if weights is not None:
        artfid = calculate_artfid(real_features, generated_features, weights)

    return fid, artfid

## Experiments

### edges2shoes_shoe

In [ ]:
import os
import subprocess
# Generating images with the style image
folder_path = "MUNIT-Enhanced/datasets/edges2shoes" # input
files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
print(files)
folder_path = "MUNIT-Enhanced/Evaluation/output/Uni-modal/style/shoes" # style
files2 = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
print(files2)
output_path = "MUNIT-Enhanced/Evaluation/output/Uni-modal/shoe/"
# tags = ["tag1", "tag2", "tag3"]  # Replace with your desired tags
for tag in files:
  for tag2 in files2:
    var1 = tag.split(".")[0].split("/")[-1]
    var2 = tag2.split(".")[0].split("/")[-1]
    var3 = "_".join([var1, var2])
    # os.makedirs(os.path.join(output_path, var3))
    output_path_ = "".join([output_path, var3])
    # command = f"!python MUNIT-Enhanced/test.py --config MUNIT-Enhanced/configs/edges2handbags_folder.yaml --input {tag} --output_folder {output_path_} --checkpoint MUNIT-Enhanced/models/edges2handbags.pt --a2b 1 --style {tag2}"
    command = [
        "python", "MUNIT-Enhanced/test.py",
        "--config", "MUNIT-Enhanced/configs/edges2shoes_folder.yaml",
        "--input", tag,
        "--output_folder", output_path_,
        "--checkpoint", "MUNIT-Enhanced/models/edges2shoes.pt",
        "--a2b", "1",
        "--style", tag2
    ]
    print(f"Running: {command}")
    # get_ipython().system(command)  # Executes the command
    # subprocess.run(command, check=True)
    try:
        result = subprocess.run(
            command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
        )
        print(f"Command output:\n{result.stdout}")
    except subprocess.CalledProcessError as e:
        print(f"Command failed with error:\n{e.stderr}")

# !python test.py --config configs/summer2winter_yosemite256_folder.yaml --input Evaluation/inputs/summer2winter/Copy\ of\ 2011-06-03\ 15_29_50.jpg --output_folder Evaluation/output/summer2winter/Copy\ of\ 2011-06-03\ 15_29_50 --checkpoint models/summer2winter_model.pt --a2b 1

['MUNIT-Enhanced/datasets/edges2shoes/Copy of image2.jpg', 'MUNIT-Enhanced/datasets/edges2shoes/Copy of image1.jpg', 'MUNIT-Enhanced/datasets/edges2shoes/Copy of image3.jpg', 'MUNIT-Enhanced/datasets/edges2shoes/Copy of 1.jpg']
['MUNIT-Enhanced/Evaluation/output/Uni-modal/style/shoes/Copy of image5.jpg', 'MUNIT-Enhanced/Evaluation/output/Uni-modal/style/shoes/Copy of image3.jpg', 'MUNIT-Enhanced/Evaluation/output/Uni-modal/style/shoes/Copy of image4.jpg', 'MUNIT-Enhanced/Evaluation/output/Uni-modal/style/shoes/Copy of 35.png', 'MUNIT-Enhanced/Evaluation/output/Uni-modal/style/shoes/Copy of 18.png', 'MUNIT-Enhanced/Evaluation/output/Uni-modal/style/shoes/Copy of 20.png', 'MUNIT-Enhanced/Evaluation/output/Uni-modal/style/shoes/Copy of 21.png', 'MUNIT-Enhanced/Evaluation/output/Uni-modal/style/shoes/Copy of 00.png', 'MUNIT-Enhanced/Evaluation/output/Uni-modal/style/shoes/Copy of 26.png', 'MUNIT-Enhanced/Evaluation/output/Uni-modal/style/shoes/Copy of 32.png', 'MUNIT-Enhanced/Evaluation/ou

In [ ]:
# Comparing Generated image with the content image using multimodality
folder_path = "MUNIT/Evaluation/inputs/edges_shoes"
files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
print(files)
folder_path2 = "MUNIT/Evaluation/output/Multi-modal/edges2shoes_edge"
files2 = [os.path.join(folder_path2, f) for f in os.listdir(folder_path2) if os.path.isfile(os.path.join(folder_path2, f))]
print(files2)
print(len(files2))
input_images = files  # Paths to input shoe images
generated_images = [files2]
lpips_score=0
fid=0
artfid=0
for i in range(10):
  temp = lpips_function(input_images[0],generated_images[0][i])
  lpips_score += temp
fid, artfid = compute_fid_and_artfid( input_images*len(files2),generated_images[0], weights=lpips_score/len(files2))
# Calculate Inception Score
cis_score, mean_is, std_is = calculate_cis(input_images, generated_images, inception_model,num_splits=1)
# print(f"Average LPIPS score = {lpips_score/len(files2)}")
print(f"Average LPIPS score = {lpips_score/len(files2)}, Average FID score = {fid}, Average ArtFid score = {artfid}")
print(f"Inception Score: Mean = {mean_is}, Std = {std_is}")
print(f"Conditional Inception Score (CIS): {cis_score}")

['MUNIT/Evaluation/inputs/edges_shoes/edges2shoes_edge.jpg']
['MUNIT/Evaluation/output/Multi-modal/edges2shoes_edge/output003.jpg', 'MUNIT/Evaluation/output/Multi-modal/edges2shoes_edge/output002.jpg', 'MUNIT/Evaluation/output/Multi-modal/edges2shoes_edge/input.jpg', 'MUNIT/Evaluation/output/Multi-modal/edges2shoes_edge/output000.jpg', 'MUNIT/Evaluation/output/Multi-modal/edges2shoes_edge/output009.jpg', 'MUNIT/Evaluation/output/Multi-modal/edges2shoes_edge/output008.jpg', 'MUNIT/Evaluation/output/Multi-modal/edges2shoes_edge/output001.jpg', 'MUNIT/Evaluation/output/Multi-modal/edges2shoes_edge/output006.jpg', 'MUNIT/Evaluation/output/Multi-modal/edges2shoes_edge/output004.jpg', 'MUNIT/Evaluation/output/Multi-modal/edges2shoes_edge/output007.jpg', 'MUNIT/Evaluation/output/Multi-modal/edges2shoes_edge/output005.jpg']
11
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.10/dist-packages/lpips/weights/v0.1/alex.pth
Setting u

### edges2handbags

In [ ]:
import os
import subprocess
# Generating image with the style image explicitly mentioned
folder_path = "MUNIT-Enhanced/datasets/edges2handbags/testA" # input
files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
print(files)
folder_path = "MUNIT-Enhanced/Evaluation/output/Uni-modal/style/handbag" # style
files2 = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
print(files2)
output_path = "MUNIT-Enhanced/Evaluation/output/Uni-modal/handbag/"
for tag in files:
  for tag2 in files2:
    var1 = tag.split(".")[0].split("/")[-1]
    var2 = tag2.split(".")[0].split("/")[-1]
    var3 = "_".join([var1, var2])
    output_path_ = "".join([output_path, var3])
    command = [
        "python", "MUNIT-Enhanced/test.py",
        "--config", "MUNIT-Enhanced/configs/edges2handbags_folder.yaml",
        "--input", tag,
        "--output_folder", output_path_,
        "--checkpoint", "MUNIT-Enhanced/models/edges2handbags.pt",
        "--a2b", "1",
        "--style", tag2
    ]
    print(f"Running: {command}")

    try:
        result = subprocess.run(
            command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
        )
        print(f"Command output:\n{result.stdout}")
    except subprocess.CalledProcessError as e:
        print(f"Command failed with error:\n{e.stderr}")

['MUNIT-Enhanced/datasets/edges2handbags/testA/69_e2h.jpg', 'MUNIT-Enhanced/datasets/edges2handbags/testA/143_e2h.jpg', 'MUNIT-Enhanced/datasets/edges2handbags/testA/139_e2h.jpg', 'MUNIT-Enhanced/datasets/edges2handbags/testA/134_e2h.jpg', 'MUNIT-Enhanced/datasets/edges2handbags/testA/100_e2h.jpg', 'MUNIT-Enhanced/datasets/edges2handbags/testA/126_e2h.jpg', 'MUNIT-Enhanced/datasets/edges2handbags/testA/48_e2h.jpg', 'MUNIT-Enhanced/datasets/edges2handbags/testA/133_e2h.jpg', 'MUNIT-Enhanced/datasets/edges2handbags/testA/90_e2h.jpg', 'MUNIT-Enhanced/datasets/edges2handbags/testA/33_e2h.jpg', 'MUNIT-Enhanced/datasets/edges2handbags/testA/6_e2h.jpg', 'MUNIT-Enhanced/datasets/edges2handbags/testA/111_e2h.jpg', 'MUNIT-Enhanced/datasets/edges2handbags/testA/62_e2h.jpg', 'MUNIT-Enhanced/datasets/edges2handbags/testA/92_e2h.jpg', 'MUNIT-Enhanced/datasets/edges2handbags/testA/441_e2h.jpg', 'MUNIT-Enhanced/datasets/edges2handbags/testA/306_e2h.jpg', 'MUNIT-Enhanced/datasets/edges2handbags/testA/2

In [ ]:
# edges2handbags
# Comparing Generated image with the content image
import itertools
files = []
files2 = []
cis_files = []
folder_path = "MUNIT/Evaluation/output/Multi-modal/edges_handbags"
folders = [name for name in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, name))]
for folder in folders:
  folder = "/".join([folder_path, folder])
  temp = [os.path.join(folder, f) for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]
  for i, t in enumerate(temp):
    var = t.split(".")[0].split("/")[-1]
    if var == 'input':
      files.extend([temp[i]]*(len(temp)-1))
      cis_files.append(temp[i])
      del temp[i]
  files2.append(temp)
print(files2, sum(len(sublist) for sublist in files2))

print(files, len( files))
print(cis_files, len( cis_files))
input_images = files  # Paths to input shoe images
generated_images = files2
lpips_score=0
fid=0
artfid=0
for j in range(len(input_images)):
  temp = lpips_function(input_images[j],list(itertools.chain(*generated_images))[j])
  lpips_score += temp
fid, artfid = compute_fid_and_artfid( input_images,list(itertools.chain(*generated_images)), weights=lpips_score/len(files))

cis_score, mean_is, std_is = calculate_cis(cis_files, generated_images, inception_model,num_splits=1)
print(f"Average LPIPS score = {lpips_score/len(files)}, Average FID score = {fid}, Average ArtFid score = {artfid}")
print(f"Inception Score: Mean = {mean_is}, Std = {std_is}")
print(f"Conditional Inception Score (CIS): {cis_score}")

[['MUNIT/Evaluation/output/Multi-modal/edges_handbags/Copy of 00006/output000.jpg', 'MUNIT/Evaluation/output/Multi-modal/edges_handbags/Copy of 00006/output001.jpg', 'MUNIT/Evaluation/output/Multi-modal/edges_handbags/Copy of 00006/output002.jpg', 'MUNIT/Evaluation/output/Multi-modal/edges_handbags/Copy of 00006/output003.jpg', 'MUNIT/Evaluation/output/Multi-modal/edges_handbags/Copy of 00006/output004.jpg', 'MUNIT/Evaluation/output/Multi-modal/edges_handbags/Copy of 00006/output005.jpg', 'MUNIT/Evaluation/output/Multi-modal/edges_handbags/Copy of 00006/output006.jpg', 'MUNIT/Evaluation/output/Multi-modal/edges_handbags/Copy of 00006/output007.jpg', 'MUNIT/Evaluation/output/Multi-modal/edges_handbags/Copy of 00006/output008.jpg', 'MUNIT/Evaluation/output/Multi-modal/edges_handbags/Copy of 00006/output009.jpg'], ['MUNIT/Evaluation/output/Multi-modal/edges_handbags/Copy of 00002/output000.jpg', 'MUNIT/Evaluation/output/Multi-modal/edges_handbags/Copy of 00002/output001.jpg', 'MUNIT/Evalu

### summer 2 winter

In [ ]:
# Comparing Generated image with the content image
import itertools
files = []
files2 = []
cis_files = []
folder_path = "MUNIT/Evaluation/output/Multi-modal/summer2winter"
folders = [name for name in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, name))]
for folder in folders:
  folder = "/".join([folder_path, folder])
  temp = [os.path.join(folder, f) for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]
  for i, t in enumerate(temp):
    var = t.split(".")[0].split("/")[-1]
    if var == 'input':
      files.extend([temp[i]]*(len(temp)-1))
      cis_files.append(temp[i])
      del temp[i]

    # else:
  files2.append(temp)
print(files2, sum(len(sublist) for sublist in files2))

# expanded_list = [x for x in files for _ in range(3)]
print(files, len( files))
print(cis_files, len( cis_files))
input_images = files  # Paths to input shoe images
generated_images = files2
flat_gen_img = list(itertools.chain(*generated_images))
lpips_score=0
fid=0
artfid=0
for j in range(len(input_images)):
  # for k in range(len(cis_files)):
  #   for i in range(len(generated_images[0])):
  temp = lpips_function(input_images[j],flat_gen_img[j])
  lpips_score += temp
fid, artfid = compute_fid_and_artfid( input_images,flat_gen_img, weights=lpips_score/len(files))

cis_score, mean_is, std_is = calculate_cis(cis_files, generated_images, inception_model,num_splits=1)
# print(f"Average LPIPS score = {lpips_score/len(files2)}")
print(f"Average LPIPS score = {lpips_score/len(files)}, Average FID score = {fid}, Average ArtFid score = {artfid}")
print(f"Inception Score: Mean = {mean_is}, Std = {std_is}")
print(f"Conditional Inception Score (CIS): {cis_score}")

[['MUNIT/Evaluation/output/Multi-modal/summer2winter/Copy of summer2winter_input/output000.jpg', 'MUNIT/Evaluation/output/Multi-modal/summer2winter/Copy of summer2winter_input/output001.jpg', 'MUNIT/Evaluation/output/Multi-modal/summer2winter/Copy of summer2winter_input/output002.jpg', 'MUNIT/Evaluation/output/Multi-modal/summer2winter/Copy of summer2winter_input/output003.jpg', 'MUNIT/Evaluation/output/Multi-modal/summer2winter/Copy of summer2winter_input/output004.jpg', 'MUNIT/Evaluation/output/Multi-modal/summer2winter/Copy of summer2winter_input/output005.jpg', 'MUNIT/Evaluation/output/Multi-modal/summer2winter/Copy of summer2winter_input/output006.jpg', 'MUNIT/Evaluation/output/Multi-modal/summer2winter/Copy of summer2winter_input/output007.jpg', 'MUNIT/Evaluation/output/Multi-modal/summer2winter/Copy of summer2winter_input/output008.jpg', 'MUNIT/Evaluation/output/Multi-modal/summer2winter/Copy of summer2winter_input/output009.jpg'], ['MUNIT/Evaluation/output/Multi-modal/summer2wi

### cat to dog

In [ ]:
# cat to dog
# Comparing Generated image with the content image
import itertools
files = []
files2 = []
cis_files = []
folder_path = "MUNIT/Evaluation/output/Multi-modal/housecat2dog"
folders = [name for name in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, name))]
for folder in folders:
  folder = "/".join([folder_path, folder])
  temp = [os.path.join(folder, f) for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]
  for i, t in enumerate(temp):
    var = t.split(".")[0].split("/")[-1]
    if var == 'input':
      files.extend([temp[i]]*(len(temp)-1))
      cis_files.append(temp[i])
      del temp[i]

    # else:
  files2.append(temp)
print(files2, sum(len(sublist) for sublist in files2))

# expanded_list = [x for x in files for _ in range(3)]
print(files, len( files))
print(cis_files, len( cis_files))
input_images = files  # Paths to input shoe images
generated_images = files2
flat_gen_img = list(itertools.chain(*generated_images))
lpips_score=0
fid=0
artfid=0
for j in range(len(input_images)):
  # for k in range(len(cis_files)):
  #   for i in range(len(generated_images[0])):
  temp = lpips_function(input_images[j],flat_gen_img[j])
  lpips_score += temp
fid, artfid = compute_fid_and_artfid( input_images,flat_gen_img, weights=lpips_score/len(files))

cis_score, mean_is, std_is = calculate_cis(cis_files, generated_images, inception_model,num_splits=1)
# print(f"Average LPIPS score = {lpips_score/len(files2)}")
print(f"Average LPIPS score = {lpips_score/len(files)}, Average FID score = {fid}, Average ArtFid score = {artfid}")
print(f"Inception Score: Mean = {mean_is}, Std = {std_is}")
print(f"Conditional Inception Score (CIS): {cis_score}")

[['MUNIT/Evaluation/output/Multi-modal/housecat2dog/Copy of pixabay_cat_004663/output000.jpg', 'MUNIT/Evaluation/output/Multi-modal/housecat2dog/Copy of pixabay_cat_004663/output001.jpg', 'MUNIT/Evaluation/output/Multi-modal/housecat2dog/Copy of pixabay_cat_004663/output002.jpg', 'MUNIT/Evaluation/output/Multi-modal/housecat2dog/Copy of pixabay_cat_004663/output003.jpg', 'MUNIT/Evaluation/output/Multi-modal/housecat2dog/Copy of pixabay_cat_004663/output004.jpg', 'MUNIT/Evaluation/output/Multi-modal/housecat2dog/Copy of pixabay_cat_004663/output005.jpg', 'MUNIT/Evaluation/output/Multi-modal/housecat2dog/Copy of pixabay_cat_004663/output006.jpg', 'MUNIT/Evaluation/output/Multi-modal/housecat2dog/Copy of pixabay_cat_004663/output007.jpg', 'MUNIT/Evaluation/output/Multi-modal/housecat2dog/Copy of pixabay_cat_004663/output008.jpg', 'MUNIT/Evaluation/output/Multi-modal/housecat2dog/Copy of pixabay_cat_004663/output009.jpg'], ['MUNIT/Evaluation/output/Multi-modal/housecat2dog/Copy of pixabay_

## Appendix

In [ ]:
# Run an iteration to get 10 outputs from a single input image
!python test.py --config configs/summer2winter_yosemite256_folder.yaml --input Evaluation/inputs/summer2winter/Copy\ of\ 2011-06-03\ 15_29_50.jpg --output_folder Evaluation/output/summer2winter/Copy\ of\ 2011-06-03\ 15_29_50 --checkpoint models/summer2winter_model.pt --a2b 1

In [ ]:
# Run an iteration to get 1 output from a single input image mentioning the style explicitly
!python MUNIT-Enhanced/test.py --config MUNIT-Enhanced/configs/edges2handbags_folder.yaml --input MUNIT-Enhanced/datasets/edges2handbags/testA/69_e2h.jpg --output_folder MUNIT-Enhanced/Evaluation/output/Uni-modal/handbag/1 --checkpoint MUNIT-Enhanced/models/edges2handbags.pt --a2b 1 --style MUNIT-Enhanced/Evaluation/output/Uni-modal/style/handbag/Copy\ of\ output003.jpg

/content/drive/MyDrive/Classes/10-623GenAI/project/MUNIT-Enhanced/test.py:54: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(opts.checkpoint)
/content

In [ ]:
# Helper code functions
import os
folder_path = "MUNIT/Evaluation/inputs/summer2winter"
files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
print(files)

['MUNIT/Evaluation/inputs/summer2winter/Copy of summer2winter_input.jpg', 'MUNIT/Evaluation/inputs/summer2winter/Copy of 2011-06-09 12_02_20.jpg', 'MUNIT/Evaluation/inputs/summer2winter/Copy of 2011-06-20 08_47_21.jpg', 'MUNIT/Evaluation/inputs/summer2winter/Copy of 2011-06-03 21_27_20.jpg', 'MUNIT/Evaluation/inputs/summer2winter/Copy of 2011-06-14 23_29_30.jpg', 'MUNIT/Evaluation/inputs/summer2winter/Copy of 2011-06-26 19_33_40.jpg', 'MUNIT/Evaluation/inputs/summer2winter/Copy of 2011-06-04 19_38_11.jpg', 'MUNIT/Evaluation/inputs/summer2winter/Copy of 2011-06-27 23_17_41.jpg', 'MUNIT/Evaluation/inputs/summer2winter/Copy of 2011-06-03 15_29_50.jpg', 'MUNIT/Evaluation/inputs/summer2winter/Copy of 2011-06-29 20_57_51.jpg']


In [ ]:
# Helper code functions
base_folder = "MUNIT/Evaluation/output/summer2winter/"
for f in files:
  var = f.split(".")[0].split("/")[-1]
  os.makedirs(os.path.join(base_folder, var))

In [ ]:
# Helper code functions
import os
files = []
files2 = []
cis_files = []
folder_path = "MUNIT/Evaluation/output/Multi-modal/edges_handbags"
folders = [name for name in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, name))]
for folder in folders:
  folder = "/".join([folder_path, folder])
  temp = [os.path.join(folder, f) for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]
  for i, t in enumerate(temp):
    var = t.split(".")[0].split("/")[-1]
    if var == 'input':
      files.extend([temp[i]]*(len(temp)-1))
      cis_files.append(temp[i])
      del temp[i]

    # else:
  files2.append(temp)
print(files2, sum(len(sublist) for sublist in files2))

# expanded_list = [x for x in files for _ in range(3)]
print(files, len( files))
print(cis_files, len( cis_files))

[['MUNIT/Evaluation/output/Multi-modal/edges_handbags/Copy of 00006/output000.jpg', 'MUNIT/Evaluation/output/Multi-modal/edges_handbags/Copy of 00006/output001.jpg', 'MUNIT/Evaluation/output/Multi-modal/edges_handbags/Copy of 00006/output002.jpg', 'MUNIT/Evaluation/output/Multi-modal/edges_handbags/Copy of 00006/output003.jpg', 'MUNIT/Evaluation/output/Multi-modal/edges_handbags/Copy of 00006/output004.jpg', 'MUNIT/Evaluation/output/Multi-modal/edges_handbags/Copy of 00006/output005.jpg', 'MUNIT/Evaluation/output/Multi-modal/edges_handbags/Copy of 00006/output006.jpg', 'MUNIT/Evaluation/output/Multi-modal/edges_handbags/Copy of 00006/output007.jpg', 'MUNIT/Evaluation/output/Multi-modal/edges_handbags/Copy of 00006/output008.jpg', 'MUNIT/Evaluation/output/Multi-modal/edges_handbags/Copy of 00006/output009.jpg'], ['MUNIT/Evaluation/output/Multi-modal/edges_handbags/Copy of 00002/output000.jpg', 'MUNIT/Evaluation/output/Multi-modal/edges_handbags/Copy of 00002/output001.jpg', 'MUNIT/Evalu